In [1]:

import json
import shutil
import os
import pickle
from callback import MultipleClassAUROC, MultiGPUModelCheckpoint
from configparser import ConfigParser
from generator import AugmentedImageSequence
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import multi_gpu_model
from utility import get_sample_counts
from weights import get_class_weights
from augmenter import augmenter
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import time
import tensorflow.keras.initializers
import statistics
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Input, GaussianNoise
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_radam import RAdam
from sklearn.metrics import confusion_matrix

%load_ext tensorboard

from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

import numpy as np

#print("TensorFlow version: ", tf.__version__)
#assert version.parse(tf.__version__).release[0] >= 2, \
#    "This notebook requires TensorFlow 2.0 or above."


Using TensorFlow backend.


In [2]:
import gc

In [3]:
import random 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
# UPDATED: import from tensorflow.keras instead of keras
from tensorflow.keras import layers, optimizers, losses, metrics
import gc
import tensorflow_addons as tfa

In [4]:
oneClass = "Emphysema"

In [5]:
config_file = "./config.ini"
cp = ConfigParser()
cp.read(config_file)

    # default config
output_dir = cp["DEFAULT"].get("output_dir")
image_source_dir = cp["DEFAULT"].get("image_source_dir")
base_model_name = cp["DEFAULT"].get("base_model_name")
class_names = cp["DEFAULT"].get("class_names").split(",")

    # train config
use_base_model_weights = cp["TRAIN"].getboolean("use_base_model_weights")
use_trained_model_weights = cp["TRAIN"].getboolean("use_trained_model_weights")
use_best_weights = cp["TRAIN"].getboolean("use_best_weights")
output_weights_name = cp["TRAIN"].get("output_weights_name")
epochs = cp["TRAIN"].getint("epochs")
batch_size = cp["TRAIN"].getint("batch_size")
initial_learning_rate = cp["TRAIN"].getfloat("initial_learning_rate")
generator_workers = cp["TRAIN"].getint("generator_workers")
image_dimension = cp["TRAIN"].getint("image_dimension")
train_steps = cp["TRAIN"].get("train_steps")
patience_reduce_lr = cp["TRAIN"].getint("patience_reduce_lr")
min_lr = cp["TRAIN"].getfloat("min_lr")
validation_steps = cp["TRAIN"].get("validation_steps")
positive_weights_multiply = cp["TRAIN"].getfloat("positive_weights_multiply")
dataset_csv_dir = cp["TRAIN"].get("dataset_csv_dir")

In [6]:
def focal_loss(gamma=1.0, alpha=0.5):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [7]:
from tensorflow.keras.losses import Huber

In [8]:
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(tf.cast(y_pred,tf.float32) - tf.cast(y_true,tf.float32)), axis=-1))

In [9]:
def construct_network1(dropout=0.08425517073874295, neuronPct=0.1767547775828121, neuronShrink=0.33180474398878285):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat1'))
    model.add(Dense(neuronCount,name='dense1'))
    model.add(Activation('relu',name='relu1'))
    model.add(Dropout(dropout, name='dropout1'))
    model.add(Dense(14, activation='sigmoid',name='midLayer1')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer1')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer1')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [10]:
def construct_network2(dropout=0.15672137551441198, neuronPct=0.2197894476507525, neuronShrink=0.3803316528497302, noisePct=0.282563134185142):
    # We start with some percent of 5000 starting neurons on the first hidden layer.
    neuronCount = int(neuronPct * 5000)
    # Construct neural network
    neuronCount = neuronCount * neuronShrink
    model = Sequential()
    model.add(Input(shape=(1,1536)))
    model.add(Flatten(name='flat2'))
    model.add(Dense(neuronCount,name='dense2'))
    model.add(GaussianNoise(noisePct))
    model.add(Activation('relu',name='relu2'))
    model.add(Dropout(dropout, name='dropout2'))
    model.add(Dense(14, activation='sigmoid',name='midLayer2')) # Output
    weights_path = None
    if weights_path is not None:
        print(f"load model weights_path: {weights_path}")
        model.load_weights(weights_path)
    #model.layers.pop()
    dr = model.layers[-2].output
    model.trainable = False
    left = Dense(14, activation="sigmoid", name='leftLayer2')(dr)
    right = Dense(14, activation="sigmoid", name='rightLayer2')(dr)
    model = Model(model.input, [left,model.output,right])
    #model = Model(model.input, model.output)
    return model

In [11]:
def construct_network(model1,model2):
    model = Model([model1.input,model2.input], [model1.output,model2.output])
    return model 

In [12]:
# compute steps
train_counts, train_pos_counts = get_sample_counts(output_dir, "train"+oneClass, class_names)
dev_counts, _ = get_sample_counts(output_dir, "dev"+oneClass, class_names)
    
if train_steps == "auto":
    train_steps = int(train_counts / batch_size)
else:
    try:
        train_steps = int(train_steps)
    except ValueError:
        raise ValueError(f"""train_steps: {train_steps} is invalid,please use 'auto' or integer.""")
    print(f"** train_steps: {train_steps} **")

if validation_steps == "auto":
    validation_steps = int(dev_counts / batch_size)
else:
    try:
        validation_steps = int(validation_steps)
    except ValueError:
        raise ValueError(f"""validation_steps: {validation_steps} is invalid,please use 'auto' or integer.""")
        print(f"** validation_steps: {validation_steps} **")

        # compute class weights
keras.backend.clear_session()
print("** compute class weights from training data **")
class_weights = get_class_weights(train_counts,train_pos_counts,multiply=positive_weights_multiply,)
print("** class_weights **")
print(class_weights)
#print(str(train_steps))
#print(str(train_counts))
#print(str(batch_size))

** compute class weights from training data **
316: 1799
38: 1799
265: 1799
313: 1799
141: 1799
71: 1799
16: 1799
539: 1799
79: 1799
26: 1799
1799: 1799
25: 1799
107: 1799
0: 1799
** class_weights **
[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 1}]


In [13]:
test_steps = cp["TEST"].get("test_steps")
test_counts, _ = get_sample_counts(output_dir, "test", class_names)

if test_steps == "auto":
    test_steps = int(test_counts / batch_size)
else:
    try:
        test_steps = int(test_steps)
    except ValueError:
        raise ValueError(f"""test_steps: {test_steps} is invalid,please use 'auto' or integer.""")
        
print(f"** test_steps: {test_steps} **")

** test_steps: 22433 **


In [14]:
train_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "train"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=train_steps,
        )
validation_sequence = AugmentedImageSequence(
            dataset_csv_file=os.path.join(output_dir, "dev"+oneClass+".csv"),
            class_names=class_names,
            source_image_dir=image_source_dir,
            batch_size=batch_size,
            target_size=(image_dimension, image_dimension),
            augmenter=augmenter,
            steps=validation_steps,
            shuffle_on_epoch_end=False,
)

test_sequence = AugmentedImageSequence(
        dataset_csv_file=os.path.join(output_dir, "test.csv"),
        class_names=class_names,
        source_image_dir=image_source_dir,
        batch_size=batch_size,
        target_size=(image_dimension, image_dimension),
        augmenter=None,
        steps=test_steps,
        shuffle_on_epoch_end=False,
    )

In [15]:
def optimize_network(lr):
    gc.collect()
      # Define the Keras TensorBoard callback.
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    start_time = time.time()
    model1 = construct_network1()
    model2 = construct_network2()
    
    optimizer = SGD(lr=initial_learning_rate)
    
    alpha = 0.9340456763831478
    gamma = 1.4195808780694898
    model1.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss})

    alpha = 0.7297456293468533
    gamma = 1.2700405014991505
    model2.compile(optimizer=optimizer,loss={'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})
  
    model = construct_network(model1=model1,model2=model2)
    model.compile(optimizer=optimizer,loss={'leftLayer1':tf.keras.losses.Huber(),'midLayer1':focal_loss(gamma=gamma,alpha=alpha),'rightLayer1':euclidean_distance_loss,'leftLayer2':tf.keras.losses.Huber(),'midLayer2':focal_loss(gamma=gamma,alpha=alpha),'rightLayer2':euclidean_distance_loss})

    output_weights_path = os.path.join(output_dir,  str(lr)+"_"+output_weights_name)
    
    print(f"** set output weights path to: {output_weights_path} **")
                  
    
                  
    checkpoint = ModelCheckpoint(
                 output_weights_path,
                 save_weights_only=True,
                 save_best_only=True,
                 verbose=1,
            )
    start_time = time.time()
  
    model.summary()
  
    callbacks = [
            checkpoint,
            #keras.callbacks.TensorBoard(log_dir=logdir),
            #TensorBoard(log_dir=os.path.join(output_dir, "logs"), batch_size=batch_size),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_reduce_lr,
                              verbose=1, mode="min", min_lr=min_lr), 
            EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto', restore_best_weights=True)
    ]
    
    
    history = model.fit_generator(
            generator=train_sequence,
            steps_per_epoch=train_steps,
            epochs=epochs,
            validation_data=validation_sequence,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=[class_weights,class_weights,class_weights,class_weights,class_weights,class_weights],
            workers=generator_workers,
            shuffle=False,
        )
        
    y_hat = model.predict_generator(test_sequence, verbose=1)
    y = test_sequence.get_y_true()
    
    test_log_path = os.path.join(output_dir, str(lr)+"_"+"test.log")
    print(f"** write log to {test_log_path} **")
    aurocs = []
    auprcs = []
    precision = dict()
    recall = dict()
    threshold = dict()
    with open(test_log_path, "w") as f:
        for k in range(6):
            for i in range(len(class_names)):
                 if(class_names[i] == str(oneClass)):
                
                    try:
                        score = roc_auc_score(y[:, i], y_hat[k][:, i])
                        precision[i], recall[i], threshold[i] = precision_recall_curve(y[:, i], y_hat[k][:, i])
                        tmp = auc(recall[i], precision[i])
                        aurocs.append(score)
                        auprcs.append(tmp) 
                    except ValueError:
                        score = 0
               
                    print(f"auroc {str(k)+class_names[i]}: {score}\n")
                    print(f"auprc {str(k)+class_names[i]}: {tmp}\n")
                    f.write(f"auroc {str(k)+class_names[i]}: {score}\n")
                    f.write(f"auprc {str(k)+class_names[i]}: {tmp}\n")
        
        mean_auroc = np.mean(aurocs)
        mean_auprc = float(np.mean(auprcs))
        f.write("-------------------------\n")
        f.write(f"mean auroc: {mean_auroc}\n")
        print(f"mean auroc: {mean_auroc}\n")
        f.write(f"mean auprc: {mean_auprc}\n")
        print(f"mean auprc: {mean_auprc}\n")
        
        max_auroc = np.max(aurocs)
        max_auprc = float(np.max(auprcs))
        f.write("-------------------------\n")
        f.write(f"max auroc: {max_auroc}\n")
        print(f"max auroc: {max_auroc}\n")
        f.write(f"max auprc: {max_auprc}\n")
        print(f"max auprc: {max_auprc}\n")
    
    keras.backend.clear_session()
    time_took = time.time() - start_time
    
    return max_auroc, time_took
    


In [16]:
step = np.arange(0.009, 0.0151, 0.001)
maxi = []
for k in np.nditer(step):
    opn, daTime = optimize_network(k)
    print(daTime)
    maxi.append(opn)




** set output weights path to: ./experiments/0.009_weights.h5 **
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 1, 1536)]    0                                            
__________________________________________________________________________________________________
flat2 (Flatten)                 (None, 1536)         0           input_2[0][0]                    
__________________________________________________________________________________________________
flat1 (Flatten)                 (None, 1536)         0           input_1[0][0]                    
___________________________

Epoch 6/11
1776/1799 [============================>.] - ETA: 0s - loss: 4.9297 - leftLayer1_loss: 0.1018 - midLayer1_loss: 1.3594 - rightLayer1_loss: 1.0628 - leftLayer2_loss: 0.0761 - midLayer2_loss: 1.4467 - rightLayer2_loss: 0.8830
Epoch 00006: val_loss improved from 5.09356 to 5.00260, saving model to ./experiments/0.009_weights.h5
1799/1799 [==============================] - 4s 2ms/step - loss: 4.9322 - leftLayer1_loss: 0.1018 - midLayer1_loss: 1.3602 - rightLayer1_loss: 1.0635 - leftLayer2_loss: 0.0760 - midLayer2_loss: 1.4463 - rightLayer2_loss: 0.8843 - val_loss: 5.0026 - val_leftLayer1_loss: 0.0994 - val_midLayer1_loss: 1.3554 - val_rightLayer1_loss: 1.0464 - val_leftLayer2_loss: 0.0899 - val_midLayer2_loss: 1.3695 - val_rightLayer2_loss: 1.0420
Epoch 7/11
1778/1799 [============================>.] - ETA: 0s - loss: 4.8595 - leftLayer1_loss: 0.0984 - midLayer1_loss: 1.3599 - rightLayer1_loss: 1.0190 - leftLayer2_loss: 0.0715 - midLayer2_loss: 1.4423 - rightLayer2_loss: 0.8683


  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 1799 steps, validate for 208 steps
Epoch 1/11
1788/1799 [============================>.] - ETA: 0s - loss: 6.4301 - leftLayer1_loss: 0.1271 - midLayer1_loss: 1.4459 - rightLayer1_loss: 1.7494 - leftLayer2_loss: 0.1271 - midLayer2_loss: 1.4022 - rightLayer2_loss: 1.5785
Epoch 00001: val_loss improved from inf to 6.17539, saving model to ./experiments/0.009999999999999998_weights.h5
1799/1799 [==============================] - 5s 3ms/step - loss: 6.4282 - leftLayer1_loss: 0.1270 - midLayer1_loss: 1.4461 - rightLayer1_loss: 1.7487 - leftLayer2_loss: 0.1270 - midLayer2_loss: 1.4023 - rightLayer2_loss: 1.5769 - val_loss: 6.1754 - val_leftLayer1_loss: 0.1245 - val_midLayer1_loss: 1.4402 - val_rightLayer1_loss: 1.6392 - val_leftLayer2_loss: 0.1212 - val_midLayer2_loss: 1.3491 - val_rightLayer2_loss: 1.5

1793/1799 [============================>.] - ETA: 0s - loss: 4.7958 - leftLayer1_loss: 0.0922 - midLayer1_loss: 1.4467 - rightLayer1_loss: 0.9448 - leftLayer2_loss: 0.0601 - midLayer2_loss: 1.4015 - rightLayer2_loss: 0.8505
Epoch 00011: val_loss improved from 4.90552 to 4.87328, saving model to ./experiments/0.009999999999999998_weights.h5
22433/22433 [==============================] - 27s 1ms/step
** write log to ./experiments/0.009999999999999998_test.log **
auroc 0Emphysema: 0.6268631518275852

auprc 0Emphysema: 0.035985540003138666

auroc 1Emphysema: 0.5972883553077984

auprc 1Emphysema: 0.027278643992755487

auroc 2Emphysema: 0.6362170853482418

auprc 2Emphysema: 0.04011498176718559

auroc 3Emphysema: 0.3803704904494146

auprc 3Emphysema: 0.016367750720864157

auroc 4Emphysema: 0.6569413394154265

auprc 4Emphysema: 0.037054418496181185

auroc 5Emphysema: 0.8088201373632578

auprc 5Emphysema: 0.10899340664763284

mean auroc: 0.6177500932852874

mean auprc: 0.04429912360462632

max 

Epoch 4/11
1781/1799 [============================>.] - ETA: 0s - loss: 5.2660 - leftLayer1_loss: 0.1088 - midLayer1_loss: 1.3322 - rightLayer1_loss: 1.2734 - leftLayer2_loss: 0.0921 - midLayer2_loss: 1.4889 - rightLayer2_loss: 0.9707
Epoch 00004: val_loss improved from 5.46727 to 5.27943, saving model to ./experiments/0.010999999999999998_weights.h5
1799/1799 [==============================] - 4s 2ms/step - loss: 5.2674 - leftLayer1_loss: 0.1088 - midLayer1_loss: 1.3324 - rightLayer1_loss: 1.2733 - leftLayer2_loss: 0.0921 - midLayer2_loss: 1.4894 - rightLayer2_loss: 0.9715 - val_loss: 5.2794 - val_leftLayer1_loss: 0.1060 - val_midLayer1_loss: 1.3280 - val_rightLayer1_loss: 1.2204 - val_leftLayer2_loss: 0.1008 - val_midLayer2_loss: 1.3642 - val_rightLayer2_loss: 1.1599
Epoch 5/11
1791/1799 [============================>.] - ETA: 0s - loss: 5.1190 - leftLayer1_loss: 0.1052 - midLayer1_loss: 1.3324 - rightLayer1_loss: 1.1774 - leftLayer2_loss: 0.0855 - midLayer2_loss: 1.4900 - rightLayer

...

1798/1799 [============================>.] - ETA: 0s - loss: 4.9299 - leftLayer1_loss: 0.0858 - midLayer1_loss: 1.5196 - rightLayer1_loss: 0.9237 - leftLayer2_loss: 0.0586 - midLayer2_loss: 1.4987 - rightLayer2_loss: 0.8435
Epoch 00011: val_loss improved from 4.96037 to 4.93023, saving model to ./experiments/0.011999999999999997_weights.h5
22433/22433 [==============================] - 29s 1ms/step
** write log to ./experiments/0.011999999999999997_test.log **
auroc 0Emphysema: 0.48883327616137046

auprc 0Emphysema: 0.020413442506064448

auroc 1Emphysema: 0.5372185893830769

auprc 1Emphysema: 0.028585946961821873

auroc 2Emphysema: 0.7042400268977058

auprc 2Emphysema: 0.04353344746242461

auroc 3Emphysema: 0.5220347734574413

auprc 3Emphysema: 0.022426682278625147

auroc 4Emphysema: 0.6403890256356214

auprc 4Emphysema: 0.03708744331643454

auroc 5Emphysema: 0.5388394324526701

auprc 5Emphysema: 0.02326233735013626

mean auroc: 0.5719258539979809

mean auprc: 0.02921821664591781

max 

Epoch 4/11
1776/1799 [============================>.] - ETA: 0s - loss: 5.0579 - leftLayer1_loss: 0.1101 - midLayer1_loss: 1.3148 - rightLayer1_loss: 1.1921 - leftLayer2_loss: 0.0927 - midLayer2_loss: 1.3962 - rightLayer2_loss: 0.9520
Epoch 00004: val_loss improved from 5.30402 to 5.12616, saving model to ./experiments/0.012999999999999996_weights.h5
1799/1799 [==============================] - 4s 2ms/step - loss: 5.0598 - leftLayer1_loss: 0.1101 - midLayer1_loss: 1.3150 - rightLayer1_loss: 1.1922 - leftLayer2_loss: 0.0927 - midLayer2_loss: 1.3970 - rightLayer2_loss: 0.9528 - val_loss: 5.1262 - val_leftLayer1_loss: 0.1074 - val_midLayer1_loss: 1.3007 - val_rightLayer1_loss: 1.1467 - val_leftLayer2_loss: 0.0983 - val_midLayer2_loss: 1.3475 - val_rightLayer2_loss: 1.1256
Epoch 5/11
1782/1799 [============================>.] - ETA: 0s - loss: 4.9215 - leftLayer1_loss: 0.1059 - midLayer1_loss: 1.3149 - rightLayer1_loss: 1.1062 - leftLayer2_loss: 0.0855 - midLayer2_loss: 1.3954 - rightLayer

  {'leftLayer1': '...', 'midLayer1': '...', 'rightLayer1': '...', 'leftLayer2': '...', 'midLayer2': '...', 'rightLayer2': '...'}
    to  
  ['...', '...', '...', '...', '...', '...']
Train for 1799 steps, validate for 208 steps
Epoch 1/11
1783/1799 [============================>.] - ETA: 0s - loss: 6.4861 - leftLayer1_loss: 0.1270 - midLayer1_loss: 1.4380 - rightLayer1_loss: 1.7517 - leftLayer2_loss: 0.1227 - midLayer2_loss: 1.4450 - rightLayer2_loss: 1.6017
Epoch 00001: val_loss improved from inf to 6.15807, saving model to ./experiments/0.013999999999999995_weights.h5
1799/1799 [==============================] - 5s 3ms/step - loss: 6.4836 - leftLayer1_loss: 0.1270 - midLayer1_loss: 1.4382 - rightLayer1_loss: 1.7509 - leftLayer2_loss: 0.1226 - midLayer2_loss: 1.4455 - rightLayer2_loss: 1.5994 - val_loss: 6.1581 - val_leftLayer1_loss: 0.1252 - val_midLayer1_loss: 1.4442 - val_rightLayer1_loss: 1.6385 - val_leftLayer2_loss: 0.1168 - val_midLayer2_loss: 1.3449 - val_rightLayer2_loss: 1.4

1778/1799 [============================>.] - ETA: 0s - loss: 4.8201 - leftLayer1_loss: 0.0928 - midLayer1_loss: 1.4384 - rightLayer1_loss: 0.9428 - leftLayer2_loss: 0.0576 - midLayer2_loss: 1.4444 - rightLayer2_loss: 0.8442
Epoch 00011: val_loss improved from 4.86641 to 4.83469, saving model to ./experiments/0.013999999999999995_weights.h5
22433/22433 [==============================] - 28s 1ms/step
** write log to ./experiments/0.013999999999999995_test.log **
auroc 0Emphysema: 0.5014360199137654

auprc 0Emphysema: 0.02723226693165653

auroc 1Emphysema: 0.5524422823047578

auprc 1Emphysema: 0.0243295412096405

auroc 2Emphysema: 0.745687997364713

auprc 2Emphysema: 0.04997561088376121

auroc 3Emphysema: 0.7908589110658754

auprc 3Emphysema: 0.10397263390599129

auroc 4Emphysema: 0.42198912549837286

auprc 4Emphysema: 0.018487532259084113

auroc 5Emphysema: 0.6932137686575056

auprc 5Emphysema: 0.041769588002369795

mean auroc: 0.617604684134165

mean auprc: 0.04429452886541724

max auro

Epoch 4/11
1786/1799 [============================>.] - ETA: 0s - loss: 5.1040 - leftLayer1_loss: 0.1071 - midLayer1_loss: 1.3311 - rightLayer1_loss: 1.1699 - leftLayer2_loss: 0.0900 - midLayer2_loss: 1.4537 - rightLayer2_loss: 0.9522
Epoch 00004: val_loss improved from 5.30772 to 5.13092, saving model to ./experiments/0.014999999999999994_weights.h5
1799/1799 [==============================] - 5s 3ms/step - loss: 5.1052 - leftLayer1_loss: 0.1071 - midLayer1_loss: 1.3316 - rightLayer1_loss: 1.1699 - leftLayer2_loss: 0.0900 - midLayer2_loss: 1.4540 - rightLayer2_loss: 0.9526 - val_loss: 5.1309 - val_leftLayer1_loss: 0.1050 - val_midLayer1_loss: 1.3222 - val_rightLayer1_loss: 1.1247 - val_leftLayer2_loss: 0.0991 - val_midLayer2_loss: 1.3696 - val_rightLayer2_loss: 1.1103
Epoch 5/11
1782/1799 [============================>.] - ETA: 0s - loss: 4.9728 - leftLayer1_loss: 0.1028 - midLayer1_loss: 1.3302 - rightLayer1_loss: 1.0870 - leftLayer2_loss: 0.0830 - midLayer2_loss: 1.4579 - rightLayer

In [17]:
print(np.max(maxi))

0.8607676760833729
